Types of methods to be used:
1. Cross-Correlation Analysis
2. Granger Causality Test
3. VAR Model
4. Impulse Response Analysis
5. Lagged Regression Analysis 

In [431]:
import MetaTrader5 as mt5
from datetime import datetime
import pytz
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings
warnings.filterwarnings("ignore")
mt5.initialize()
account=51127988
password="Aar2frM7"
server = 'ICMarkets-Demo'

In [358]:
def get_rates(pair1, tf, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, tf, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    pair1 = pair1.set_index(pair1['time'])
    pair1 = pair1.drop(columns = ['time','tick_volume', 'spread', 'real_volume'])
    return pair1

# Timezones for MT5 (GMT + 3)

## SYD - 21:00 TO 06:00
## TKO - 11:00 TO 09:00
## LDN - 08:00 TO 16:00
## NY - 13:00 TO 21:00

## Hypothesis 

The movements of the EURUSD and GBPUSD exchange rates can be used to predict the future direction of the USDJPY exchange rate. Specifically, if both EURUSD and GBPUSD experience an increase within a given time period (e.g., within one hour), there is a statistically significant probability that the USDJPY exchange rate will decrease in the subsequent time period.

## Method

Gather hourly data of EURUSD and GBPUD. Process the data so that the remaining data are EURUSD and GBPUSD candles that are correlated and have moved more than 10 pips.

Gather USDJPY hourly candles for each time period succeeding the new data points gathered.

Run a Pearson correlation analysis between the EURUSD-GBPUSD amended data and USDJPY data.

## Result 

No Correlations found 


In [313]:
EU = get_rates('EURUSD.a', mt5.TIMEFRAME_H1, 10000)
GU = get_rates('GBPUSD.a', mt5.TIMEFRAME_H1, 10000)
EU_returns = pd.DataFrame(10000 * (EU['close'] - EU['open']))
EU_returns.rename(columns = {0:'EU'}, inplace = True)
GU_returns = pd.DataFrame(10000 * (GU['close'] - GU['open']))
GU_returns.rename(columns = {0:'GU'}, inplace = True)
# GU_returns.rename('GU', inplace = True)
EU_returns = EU_returns[abs(EU_returns) > 10]
GU_returns = GU_returns[abs(GU_returns) > 10]

EU_GU = pd.concat([EU_returns, GU_returns], axis = 1)
EU_GU = EU_GU.dropna()
EU_GU = EU_GU.rename(columns={0: 'EURUSD', 0: 'GBPUSD'})
EU_GU["Spread"] = EU_GU['EU'] - EU_GU['GU']
EU_GU

,EU,GU,Spread
time,,,
2021-11-15 19:00:00,-35.4,-21.1,-14.3
2021-11-15 22:00:00,-25.5,-20.9,-4.6
2021-11-16 16:00:00,-18.0,-33.8,15.8
2021-11-16 21:00:00,-10.7,-10.2,-0.5
2021-11-17 05:00:00,-33.2,-17.0,-16.2
...,...,...,...
2023-06-20 12:00:00,-12.8,-17.0,4.2
2023-06-20 19:00:00,15.2,26.0,-10.8
2023-06-21 11:00:00,12.2,-16.0,28.2


In [339]:
UJ = get_rates('USDJPY.a', mt5.TIMEFRAME_H1, 10000)
UJ_returns = pd.DataFrame(UJ['close'] - UJ['open'])
UJ_returns.rename(columns = {0:'UJ'}, inplace = True)

In [355]:
# Shift the index in df2 back by one hour
UJ_returns.index = UJ_returns.index - pd.Timedelta(hours=1)

# Keep only the rows in UJ_returns that match the index in df1
UJ_returns = UJ_returns[UJ_returns.index.isin(EU_GU.index)]
UJ_returns

,UJ
time,
2021-11-19 10:00:00,-0.043
2021-11-19 13:00:00,0.056
2021-11-19 14:00:00,0.068
2021-11-19 15:00:00,-0.016
2021-11-22 16:00:00,-0.053
...,...
2023-06-15 14:00:00,-0.285
2023-06-15 15:00:00,0.030
2023-06-15 16:00:00,0.043


In [361]:
def corr_measure(pearson_corr, i):
    if pearson_corr > abs(0.5):
        print(f'Lag {i} shows a moderate correlation')
    else:
        print(f'No Correlation at {i}')

for i in range(1, 24):
    # Make a copy of the original dataframe
    UJ_returns_copy = UJ_returns.copy()

    # Shift the index in the copy back by one hour
    UJ_returns_copy.index = UJ_returns_copy.index - pd.Timedelta(hours=i)

    # Keep only the rows in UJ_returns_copy that match the index in EU_GU
    UJ_returns_copy = UJ_returns_copy[UJ_returns_copy.index.isin(EU_GU.index)]
    
    min_length = min(len(EU_GU), len(UJ_returns_copy))

    pearson_corr = np.corrcoef(EU_GU['EU'].tail(min_length), UJ_returns_copy['UJ'].tail(min_length))[0, 1]
    corr_measure(pearson_corr, i)
    # print(f'Lag {i} - {pearson_corr} for EU')
    
    pearson_corr = np.corrcoef(UJ_returns_copy['UJ'].tail(min_length), EU_GU['GU'].tail(min_length))[0, 1]
    corr_measure(pearson_corr, i)
    # print(f'Lag {i} - {pearson_corr} for GU')
    
    pearson_corr = np.corrcoef(EU_GU['EU'].tail(min_length), UJ_returns_copy['UJ'].tail(min_length))[0, 1]
    corr_measure(pearson_corr, i)
    # print(f'Lag {i} - {pearson_corr} for the spread')

No Correlation at 1
No Correlation at 1
No Correlation at 1
No Correlation at 2
No Correlation at 2
No Correlation at 2
No Correlation at 3
No Correlation at 3
No Correlation at 3
No Correlation at 4
No Correlation at 4
No Correlation at 4
No Correlation at 5
No Correlation at 5
No Correlation at 5
No Correlation at 6
No Correlation at 6
No Correlation at 6
No Correlation at 7
No Correlation at 7
No Correlation at 7
No Correlation at 8
No Correlation at 8
No Correlation at 8
No Correlation at 9
No Correlation at 9
No Correlation at 9
No Correlation at 10
No Correlation at 10
No Correlation at 10
No Correlation at 11
No Correlation at 11
No Correlation at 11
No Correlation at 12
No Correlation at 12
No Correlation at 12
No Correlation at 13
No Correlation at 13
No Correlation at 13
No Correlation at 14
No Correlation at 14
No Correlation at 14
No Correlation at 15
No Correlation at 15
No Correlation at 15
No Correlation at 16
No Correlation at 16
No Correlation at 16
No Correlation at 1

## Hypothesis

For each base currencies respective forex opening (Sydney for AUD, Tokyo for JPY etc etc), there will be a correlation of returns at a later time period within the same currency at a consistent hour, whether it seem like an arbitrary number or it be at the next session. For example, if AUDUSD returns +10 pips in the Syd open, later whether it be Tokyo, London or New York or a seemingly arbitrary period, it will return 10 or more pips within an hour. 

## Method

Gather data for AUDUSD, USDJPY, GBPUSD and EURUSD. Within each dataframe, run a correlation analysis between the opening time (Start at the hourly and go down to the minute timeframe). 


In [ ]:
## To edit
from datetime import datetime, timedelta

def sing_corr(symbol, base_hour, timeframe, bars = 100):

    correlations = {}  # Store correlations here, keyed by time string

    # Convert timeframe string to corresponding MetaTrader 5 constant
    timeframe_dict = {'M1': mt5.TIMEFRAME_M1, 'M2': mt5.TIMEFRAME_M2, 'M3': mt5.TIMEFRAME_M3, 'M5': mt5.TIMEFRAME_M5, 
                      'M10': mt5.TIMEFRAME_M10, 'M15': mt5.TIMEFRAME_M15, 'M30': mt5.TIMEFRAME_M30, 'H1': mt5.TIMEFRAME_H1, 
                      'H2': mt5.TIMEFRAME_H2, 'H3': mt5.TIMEFRAME_H3, 'H4': mt5.TIMEFRAME_H4, 'H6': mt5.TIMEFRAME_H6, 
                      'H8': mt5.TIMEFRAME_H8, 'H12': mt5.TIMEFRAME_H12, 'D1': mt5.TIMEFRAME_D1, 'W1': mt5.TIMEFRAME_W1, 
                      'MN1': mt5.TIMEFRAME_MN1}
    mt5_timeframe = timeframe_dict[timeframe]

    # Calculate the duration of the time period in hours
    if timeframe[-1] == 'M':
        duration = timedelta(minutes=int(timeframe[:-1]))
    elif timeframe[-1] == 'H':
        duration = timedelta(hours=int(timeframe[:-1]))
    elif timeframe[-1] == 'D':
        duration = timedelta(days=int(timeframe[:-1]))
    elif timeframe[-1] == 'W':
        duration = timedelta(weeks=int(timeframe[:-1]))
    else: # 'MN1'
        duration = timedelta(days=int(timeframe[:-1]) * 30) # Roughly convert months to days

    # Get data for base_hour
    base_ts1 = datetime.strptime(f'{base_hour}:00', '%H:%M').time()
    base_ts2 = (datetime.combine(datetime.today(), base_ts1) + duration).time()
    base_ts1, base_ts2 = [t.strftime('%H:%M') for t in [base_ts1, base_ts2]]

    base_prices = get_rates(symbol, mt5_timeframe, 50000)
    base_prices = base_prices.between_time(base_ts1, base_ts2)
    base_prices['returns'] = base_prices['close'] - base_prices['open']

    for h in range(24):  # For each hour of the day
        if h == base_hour:  # Skip the base_hour
            continue

        ts1 = datetime.strptime(f'{h}:00', '%H:%M').time()
        ts2 = (datetime.combine(datetime.today(), ts1) + duration).time()
        ts1, ts2 = [t.strftime('%H:%M') for t in [ts1, ts2]]

        prices = get_rates(symbol, mt5_timeframe, 50000)
        prices = prices.between_time(ts1, ts2)
        prices['returns'] = prices['close'] - prices['open']

        pearson_corr = np.corrcoef(base_prices['returns'].tail(bars), prices['returns'].tail(bars))[0, 1]  # Store correlation

        if abs(pearson_corr) > 0.15:
            correlations[f'{base_ts1}-{ts1}'] = pearson_corr

    return correlations

In [454]:
from datetime import datetime, timedelta
from collections import defaultdict

def t3_sing_corr(symbol, base_hour, bars = 100):

    correlations = {}  # Store correlations here, keyed by time string
    
    ordered_correlations = {}

    # Get data for base_hour
    base_ts1 = datetime.strptime(f'{base_hour}:00', '%H:%M').time()
    base_ts2 = (datetime.combine(datetime.today(), base_ts1) + timedelta(hours=1)).time()
    base_ts1, base_ts2 = [t.strftime('%H:%M') for t in [base_ts1, base_ts2]]

    base_prices = get_rates(symbol, mt5.TIMEFRAME_H1, 50000)
    base_prices = base_prices.between_time(base_ts1, base_ts2)
    base_prices['returns'] = base_prices['close'] - base_prices['open']

    for h in range(24):  # For each hour of the day
        if h == base_hour:  # Skip the base_hour
            continue

        ts1 = datetime.strptime(f'{h}:00', '%H:%M').time()
        ts2 = (datetime.combine(datetime.today(), ts1) + timedelta(hours=1)).time()
        ts1, ts2 = [t.strftime('%H:%M') for t in [ts1, ts2]]

        prices = get_rates(symbol, mt5.TIMEFRAME_H1, 50000)
        prices = prices.between_time(ts1, ts2)
        prices['returns'] = prices['close'] - prices['open']

        pearson_corr = np.corrcoef(base_prices['returns'].tail(bars), prices['returns'].tail(bars))[0, 1]  # Store correlation

        if abs(pearson_corr) > 0.2:
            correlations[f'{base_ts1}-{ts1}'] = pearson_corr
        
        if abs(pearson_corr) > 0.4:
            ordered_correlations[f'{base_ts1}-{ts1} | 0.4 and over'] = pearson_corr
        elif abs(pearson_corr) > 0.3:
            ordered_correlations[f'{base_ts1}-{ts1} | 0.3 to 0.4'] = pearson_corr
        elif abs(pearson_corr) > 0.2:
            ordered_correlations[f'{base_ts1}-{ts1} | 0.2 to 0.3'] = pearson_corr

    return correlations, ordered_correlations

In [485]:
from collections import defaultdict
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

# Assume mini_list and get_rates are defined elsewhere.
mini_list = ['AUDUSD.a', 'CADJPY.a', 'EURCAD.a', 'EURGBP.a']

def t3_sing_corr(symbol, base_hour, bars=100):
    base_ts1 = datetime.strptime(f'{base_hour}:00', '%H:%M').time()
    base_ts2 = (datetime.combine(datetime.today(), base_ts1) + timedelta(hours=1)).time()
    base_ts1, base_ts2 = [t.strftime('%H:%M') for t in [base_ts1, base_ts2]]

    base_prices = get_rates(symbol, mt5.TIMEFRAME_H1, 50000)
    base_prices = base_prices.between_time(base_ts1, base_ts2)
    base_prices['returns'] = base_prices['close'] - base_prices['open']

    correlations = {}
    ordered_correlations = defaultdict(list)

    for h in range(24):
        if h == base_hour:
            continue

        ts1 = datetime.strptime(f'{h}:00', '%H:%M').time()
        ts2 = (datetime.combine(datetime.today(), ts1) + timedelta(hours=1)).time()
        ts1, ts2 = [t.strftime('%H:%M') for t in [ts1, ts2]]

        prices = get_rates(symbol, mt5.TIMEFRAME_H1, 50000)
        prices = prices.between_time(ts1, ts2)
        prices['returns'] = prices['close'] - prices['open']

        pearson_corr = np.corrcoef(base_prices['returns'].tail(bars), prices['returns'].tail(bars))[0, 1]

        if abs(pearson_corr) > 0.2:
            correlations[f'{base_ts1}-{ts1} | {symbol}'] = pearson_corr

        if abs(pearson_corr) > 0.4:
            ordered_correlations['0.4 and over'].append((f'{base_ts1}-{ts1}', pearson_corr, symbol))
        elif abs(pearson_corr) > 0.3:
            ordered_correlations['0.3 to 0.4'].append((f'{base_ts1}-{ts1}', pearson_corr, symbol))
        elif abs(pearson_corr) > 0.2:
            ordered_correlations['0.2 to 0.3'].append((f'{base_ts1}-{ts1}', pearson_corr, symbol))

    return correlations, ordered_correlations

correlations_all = {}
ordered_correlations_all = defaultdict(list)

for symbol in mini_list:
    for i in range(24):
        correlations, ordered_correlations = t3_sing_corr(symbol, i, bars=100)
        correlations_all.update(correlations)
        for key in ordered_correlations.keys():
            ordered_correlations_all[key].extend(ordered_correlations[key])
    print(f"{symbol} Analysed")

print(f'All Correlations:\n{correlations_all}')
print(f'Ordered Correlations:\n{dict(ordered_correlations_all)}')

AUDUSD.a Analysed
CADJPY.a Analysed
EURCAD.a Analysed
EURGBP.a Analysed
All Correlations:
{'00:00-03:00 | AUDUSD.a': -0.2556354331116688, '01:00-04:00 | AUDUSD.a': -0.20098579099278488, '01:00-08:00 | AUDUSD.a': 0.20032977151050563, '03:00-00:00 | AUDUSD.a': -0.2556354331116688, '04:00-01:00 | AUDUSD.a': -0.20098579099278488, '04:00-05:00 | AUDUSD.a': 0.25865082340865325, '05:00-04:00 | AUDUSD.a': 0.25865082340865325, '05:00-10:00 | AUDUSD.a': -0.21608498772501772, '06:00-07:00 | AUDUSD.a': 0.7538900674679854, '06:00-14:00 | AUDUSD.a': 0.2179341761758313, '07:00-06:00 | AUDUSD.a': 0.7538900674679854, '07:00-08:00 | AUDUSD.a': 0.32148805044562434, '08:00-01:00 | AUDUSD.a': 0.20032977151050563, '08:00-07:00 | AUDUSD.a': 0.32148805044562434, '10:00-05:00 | AUDUSD.a': -0.21608498772501772, '14:00-06:00 | AUDUSD.a': 0.21793417617583127, '14:00-22:00 | AUDUSD.a': 0.20062884111274998, '15:00-21:00 | AUDUSD.a': -0.21914152599099385, '16:00-18:00 | AUDUSD.a': 0.22090776928500436, '16:00-22:00 |

In [491]:
ordered_correlations_all['0.4 and over']

[('06:00-07:00', 0.7538900674679854, 'AUDUSD.a'),
 ('07:00-06:00', 0.7538900674679854, 'AUDUSD.a'),
 ('06:00-07:00', 0.6513822278423064, 'CADJPY.a'),
 ('07:00-06:00', 0.6513815088688605, 'CADJPY.a'),
 ('07:00-08:00', 0.4149373505617765, 'CADJPY.a'),
 ('08:00-07:00', 0.414775346981471, 'CADJPY.a'),
 ('06:00-07:00', 0.5132543262739686, 'EURCAD.a'),
 ('07:00-06:00', 0.5132543262739686, 'EURCAD.a'),
 ('06:00-07:00', 0.41580626372576074, 'EURGBP.a'),
 ('07:00-06:00', 0.41580626372576074, 'EURGBP.a')]

In [448]:
for i in combined_lst:
    for t in range(1,24):
        print(f'Analysing {i}')
        print(t3_sing_corr(i, t))

Analysing AUDUSD.a
({'01:00-04:00': -0.20098579099278488, '01:00-08:00': 0.20032977151050563}, {'01:00-04:00 | 0.2 to 0.3': -0.20098579099278488, '01:00-08:00 | 0.2 to 0.3': 0.20032977151050563})
Analysing AUDUSD.a
({}, {})
Analysing AUDUSD.a
({'03:00-00:00': -0.2556354331116688}, {'03:00-00:00 | 0.2 to 0.3': -0.2556354331116688})
Analysing AUDUSD.a
({'04:00-01:00': -0.20098579099278488, '04:00-05:00': 0.24032319611665304}, {'04:00-01:00 | 0.2 to 0.3': -0.20098579099278488, '04:00-05:00 | 0.2 to 0.3': 0.24032319611665304})
Analysing AUDUSD.a
({'05:00-04:00': 0.24032319611665307, '05:00-06:00': 0.32086766102180325, '05:00-10:00': -0.21580210425826193}, {'05:00-04:00 | 0.2 to 0.3': 0.24032319611665307, '05:00-06:00 | 0.3 to 0.4': 0.32086766102180325, '05:00-10:00 | 0.2 to 0.3': -0.21580210425826193})
Analysing AUDUSD.a
({'06:00-05:00': 0.32086766102180325, '06:00-07:00': 0.754338701983692, '06:00-10:00': -0.24618651396743702}, {'06:00-05:00 | 0.3 to 0.4': 0.32086766102180325, '06:00-07:0

In [420]:
from datetime import datetime, timedelta

def sing_corr(symbol, base_hour, bars = 100):

    correlations = {}  # Store correlations here, keyed by time string

    # Get data for base_hour
    base_ts1 = datetime.strptime(f'{base_hour}:00', '%H:%M').time()
    base_ts2 = (datetime.combine(datetime.today(), base_ts1) + timedelta(hours=1)).time()
    base_ts1, base_ts2 = [t.strftime('%H:%M') for t in [base_ts1, base_ts2]]

    base_prices = get_rates(symbol, mt5.TIMEFRAME_H1, 50000)
    base_prices = base_prices.between_time(base_ts1, base_ts2)
    base_prices['returns'] = base_prices['close'] - base_prices['open']

    for h in range(24):  # For each hour of the day
        if h == base_hour:  # Skip the base_hour
            continue

        ts1 = datetime.strptime(f'{h}:00', '%H:%M').time()
        ts2 = (datetime.combine(datetime.today(), ts1) + timedelta(hours=1)).time()
        ts1, ts2 = [t.strftime('%H:%M') for t in [ts1, ts2]]

        prices = get_rates(symbol, mt5.TIMEFRAME_H1, 50000)
        prices = prices.between_time(ts1, ts2)
        prices['returns'] = prices['close'] - prices['open']

        pearson_corr = np.corrcoef(base_prices['returns'].tail(bars), prices['returns'].tail(bars))[0, 1]  # Store correlation

        if abs(pearson_corr) > 0.15:
            correlations[f'{base_ts1}-{ts1}'] = pearson_corr

    return correlations

In [435]:
USD_SYMBOLS = ['AUDUSD.a', 'NZDUSD.a', 'GBPUSD.a', 'EURUSD.a', 'USDCAD.a']
GBP_SYMBOLS = ['GBPAUD.a', 'GBPNZD.a', 'GBPCAD.a']
JPY_SYMBOLS = ['AUDJPY.a', 'USDJPY.a', 'NZDJPY.a', 'GBPJPY.a', 'EURJPY.a', 'CADJPY.a']
EUR_SYMBOLS = ['EURAUD.a', 'EURNZD.a', 'EURGBP.a', 'EURJPY.a', 'EURCAD.a']

combined_lst = USD_SYMBOLS + GBP_SYMBOLS + JPY_SYMBOLS + EUR_SYMBOLS
combined_lst

['AUDUSD.a',
 'NZDUSD.a',
 'GBPUSD.a',
 'EURUSD.a',
 'USDCAD.a',
 'GBPAUD.a',
 'GBPNZD.a',
 'GBPCAD.a',
 'AUDJPY.a',
 'USDJPY.a',
 'NZDJPY.a',
 'GBPJPY.a',
 'EURJPY.a',
 'CADJPY.a',
 'EURAUD.a',
 'EURNZD.a',
 'EURGBP.a',
 'EURJPY.a',
 'EURCAD.a']

In [438]:
for i in combined_lst:
    print(f'Analysing {i}')
    print(sing_corr(i, '11'))

Analysing AUDUSD.a
{'11:00-01:00': 0.19090883667277614, '11:00-03:00': 0.20408191045270305, '11:00-04:00': 0.21509278472178386, '11:00-06:00': -0.18220361406970595, '11:00-07:00': -0.18613602569628168, '11:00-11:00': 1.0}
Analysing NZDUSD.a
{'11:00-11:00': 1.0}
Analysing GBPUSD.a
{'11:00-05:00': -0.32486594690176673, '11:00-11:00': 1.0}
Analysing EURUSD.a
{'11:00-01:00': 0.23731198973783468, '11:00-05:00': -0.15027625849040452, '11:00-11:00': 1.0, '11:00-15:00': 0.1662717930826996}
Analysing USDCAD.a
{'11:00-01:00': 0.21063423328328396, '11:00-02:00': 0.15342425305777785, '11:00-03:00': 0.19980917303967652, '11:00-07:00': -0.23771047590724032, '11:00-11:00': 1.0}
Analysing GBPAUD.a
{'11:00-04:00': 0.1838278581281314, '11:00-05:00': 0.16094319074234473, '11:00-08:00': -0.28036278672911563, '11:00-11:00': 1.0, '11:00-23:00': 0.15085366744079712}
Analysing GBPNZD.a
{'11:00-06:00': 0.15207785036076363, '11:00-11:00': 1.0, '11:00-22:00': -0.15860378445564915}
Analysing GBPCAD.a
{'11:00-08:0

In [439]:
for i in combined_lst:
    print(f'Analysing {i}')
    print(sing_corr(i, '21'))

Analysing AUDUSD.a
{'21:00-00:00': 0.15486694555075625, '21:00-21:00': 1.0}
Analysing NZDUSD.a
{'21:00-04:00': -0.183881784241488, '21:00-08:00': 0.2090279475191666, '21:00-21:00': 1.0, '21:00-23:00': -0.17205526049743014}
Analysing GBPUSD.a
{'21:00-04:00': -0.1529695082128809, '21:00-21:00': 1.0, '21:00-22:00': -0.17805151513296263}
Analysing EURUSD.a
{'21:00-15:00': -0.16720918624375414, '21:00-18:00': -0.155384777495302, '21:00-21:00': 1.0}
Analysing USDCAD.a
{'21:00-00:00': 0.17283473321861767, '21:00-12:00': 0.16124573950865106, '21:00-21:00': 1.0, '21:00-22:00': -0.2558334842881715}
Analysing GBPAUD.a
{'21:00-01:00': 0.19438553916782003, '21:00-03:00': -0.1882075048149098, '21:00-06:00': -0.16559145411146176, '21:00-09:00': -0.1526067803207042, '21:00-14:00': -0.20398488120215535, '21:00-17:00': 0.19187494160438692, '21:00-21:00': 1.0, '21:00-23:00': 0.16118873905052108}
Analysing GBPNZD.a
{'21:00-04:00': -0.19649943849746201, '21:00-12:00': -0.23910747986399122, '21:00-17:00': 0

In [440]:
for i in combined_lst:
    print(f'Analysing {i}')
    print(sing_corr(i, '8'))

Analysing AUDUSD.a
{'08:00-08:00': 1.0, '08:00-13:00': 0.18330809217353108, '08:00-23:00': -0.16556862594946883}
Analysing NZDUSD.a
{'08:00-01:00': -0.1733757564852219, '08:00-08:00': 1.0, '08:00-21:00': 0.2090279475191666}
Analysing GBPUSD.a
{'08:00-05:00': 0.23743591113204424, '08:00-06:00': 0.1546754838643972, '08:00-08:00': 1.0}
Analysing EURUSD.a
{'08:00-05:00': 0.18380171583436167, '08:00-06:00': 0.15076346266821952, '08:00-08:00': 1.0, '08:00-10:00': 0.19753350830188715, '08:00-13:00': -0.20428343382672942, '08:00-22:00': -0.15137680983952614}
Analysing USDCAD.a
{'08:00-02:00': -0.2089395065214017, '08:00-08:00': 1.0, '08:00-18:00': 0.1513382641174979}
Analysing GBPAUD.a
{'08:00-07:00': 0.15255256441992135, '08:00-08:00': 1.0, '08:00-11:00': -0.28036278672911563}
Analysing GBPNZD.a
{'08:00-04:00': 0.2395920254733738, '08:00-08:00': 0.9999999999999998, '08:00-09:00': -0.24889276511826888, '08:00-20:00': -0.1663152984049021}
Analysing GBPCAD.a
{'08:00-08:00': 1.0, '08:00-11:00': -

In [441]:
for i in combined_lst:
    print(f'Analysing {i}')
    print(sing_corr(i, '13'))

Analysing AUDUSD.a
{'13:00-05:00': -0.17610793780501488, '13:00-06:00': 0.1777649735749896, '13:00-08:00': 0.18330809217353108, '13:00-13:00': 1.0, '13:00-14:00': -0.18615202125725044, '13:00-15:00': -0.1804368757139322, '13:00-18:00': -0.21457929303499934, '13:00-19:00': 0.18038490511444394}
Analysing NZDUSD.a
{'13:00-01:00': -0.1701719717467185, '13:00-03:00': 0.26676246626079525, '13:00-13:00': 1.0, '13:00-16:00': 0.18091622606599864, '13:00-17:00': 0.21143081987063692, '13:00-19:00': 0.21979537354862808}
Analysing GBPUSD.a
{'13:00-05:00': -0.15139896592312546, '13:00-13:00': 1.0, '13:00-16:00': 0.18710817428735438}
Analysing EURUSD.a
{'13:00-01:00': -0.2221273916561418, '13:00-05:00': -0.1942906191915452, '13:00-08:00': -0.20428343382672942, '13:00-13:00': 1.0, '13:00-14:00': -0.18501102158991176, '13:00-15:00': -0.18281975952608054}
Analysing USDCAD.a
{'13:00-06:00': 0.2358635601123228, '13:00-13:00': 1.0}
Analysing GBPAUD.a
{'13:00-04:00': -0.17769686509622495, '13:00-06:00': 0.2

## Hypothesis 
Currency prices whose sessions open prior to other sessions can be a leading indicator of prices for currencies sessions not yet opened, which can be found through interdependent and time lagged analysis. For example, if AUDUSD returns positive in the first hour of the Syd open, USDJPY is expected to return positive too. 

## Method

Gather data on the first 5min to 60min bars of each open, starting with Sydney. Will base currencies against the USD Markets open in the following order:
GMT Time (MT5 is GMT +3)
- Sydney: 9PM / 21:00 (AUDUSD)
- Tokyo: 12AM / 00:00 (USDJPY)
- London: 7AM / 7:00(GBPUSD)
- New York: 1PM / 13:00 (EURUSD)

Run a Pearson Correlation and a Granger Causality method on the gathered data and the respective currencies.



In [144]:
def corr_tool(symbol1, symbol2, timeframe, bars):
    print(f'Starting Pearson Correlation Analysis for {symbol1}, {symbol2} at {timeframe} for {bars} bars.')

    pair1 = get_rates(symbol1, timeframe, bars)
    pair2 = get_rates(symbol2,  timeframe, bars)
    
    s1 = input('Start Time for X?')
    s2 = input('End time for X?')

    e1 = input('Start time for Y?')
    e2 = input('End time for Y?')

    print('Times to investigate are:'
          f'\n{s1} to {s2} and {e1} to {e2}')
    
    pair1_returns = pair1['open'] - pair1['close']
    pair2_returns = pair2['open'] - pair2['close']

    lst = []

    un_corr = []

    for i in range(50):
        pearson_corr = np.corrcoef(syd_opens['Syd Pip Return'].tail(i), tokyo_opens['Tko Pip Return'].tail(i))
        if pearson_corr[0][1] > 0.5:
            # print(f'Pearson correlation {pearson_corr[0][1]} on lag {i}')
            lst.append(i)
        else:
            un_corr.append(i)
            
    print(f'Lags that have a correlation higher than 0.5 are:'
        f'\n{lst}')
    print(f'Total Lags: {len(lst)}')

    import datetime

    day_counts = {}

    for i in lst:
        date = syd_opens.index[i]
        day_name = date.day_name()

        if day_name not in day_counts:
            day_counts[day_name] = 0

        day_counts[day_name] += 1
    print(f'Days that have correlations between AUDUSD and USDJPY of more than 0.5 are:')

    for day, count in day_counts.items():
        print(f'{day}: {count}')
        
    day_counts = {}

    for i in un_corr:
        date = syd_opens.index[i]
        day_name = date.day_name()

        if day_name not in day_counts:
            day_counts[day_name] = 0

        day_counts[day_name] += 1
    print(f'Days that do not have correlations between AUDUSD and USDJPY of more than 0.5 are:')

    for day, count in day_counts.items():
        print(f'{day}: {count}')

In [45]:
corr_tool('AUDUSD.a', 'AUDJPY.a', mt5.TIMEFRAME_H1, 50000)

Starting Pearson Correlation Analysis for AUDUSD.a, AUDJPY.a at 16385 for 50000 bars.


Start Time for AUDUSD.a? 19:00
End time for AUDUSD.a? 19:59
Start time for AUDJPY.a? 20:00
End time for AUDJPY.a 20:59


Times to investigate are:
19:00 to 19:59 and 20:00 to 20:59
Lags that have a correlation higher than 0.5 are:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Total Lags: 50
Days that have correlations between AUDUSD.a and AUDJPY.aof more than 0.5 are:
Monday: 10
Tuesday: 10
Wednesday: 10
Thursday: 10
Friday: 10
Days that do not have correlations between AUDUSD.a and AUDJPY.a of more than 0.5 are:


In [174]:
#Starting with AUDUSD, Hourly Timeframe 
AUDUSD = get_rates('AUDUSD.a', mt5.TIMEFRAME_H1, 50000)
syd_opens = AUDUSD.between_time('19:00','19:59')
syd_opens['returns'] = syd_opens['close'] - syd_opens['open']
USDJPY = get_rates('USDJPY.a', mt5.TIMEFRAME_H1, 50000)
tokyo_opens = USDJPY.between_time('20:00', '20:59')
tokyo_opens['returns'] = tokyo_opens['close'] - tokyo_opens['open']
tail = 50 
pearson_corr = np.corrcoef(syd_opens['returns'].tail(tail), tokyo_opens['returns'].tail(tail)) 
pearson_corr

array([[1.        , 0.50440698],
       [0.50440698, 1.        ]])

In [169]:
def initial_mini_corr(s1, s2, bars = 10000):
    
    
    ts1_1 = input(f'Start Time for {s1}?')
    ts1_2 = input(f'End time for {s1}?')

    ts2_1 = input(f'Start time for {s2}?')
    ts2_2 = input(f'End time for {s2}?')
    
    p1 = get_rates(s1, mt5.TIMEFRAME_H1, 50000)
    p1 = p1.between_time(ts1_1, ts1_2)
    p1['returns'] = p1['close'] - p1['open']
    p2 = get_rates(s2, mt5.TIMEFRAME_H1, 50000)
    p2 = p2.between_time(ts2_1, ts2_2)
    p2['returns'] = p2['close'] - p2['open']
    
    pearson_corr = np.corrcoef(p1['returns'].tail(tail), p2['returns'].tail(tail)) 
    return pearson_corr

In [170]:
initial_mini_corr('AUDUSD.a', 'USDJPY.a')

Start Time for AUDUSD.a? 10:00
End time for AUDUSD.a? 10:59
Start time for USDJPY.a? 13:00
End time for USDJPY.a? 13:59


array([[1.        , 0.07723649],
       [0.07723649, 1.        ]])

In [ ]:
from datetime import datetime, timedelta

def hour_mini_corr(s1, s2, bars = 100):

    correlations = {}  # Store correlations here, keyed by time string

    for h in range(24):  # For each hour of the day
        ts1_1 = datetime.strptime(f'{h}:00', '%H:%M').time()
        ts1_2 = (datetime.combine(datetime.today(), ts1_1) + timedelta(hours=1)).time()

        ts2_1 = (datetime.combine(datetime.today(), ts1_1) + timedelta(hours=1)).time()
        ts2_2 = (datetime.combine(datetime.today(), ts2_1) + timedelta(hours=1)).time()

        # Convert to strings for use in function
        ts1_1, ts1_2, ts2_1, ts2_2 = [t.strftime('%H:%M') for t in [ts1_1, ts1_2, ts2_1, ts2_2]]

        p1 = get_rates(s1, mt5.TIMEFRAME_H1, 50000)
        p1 = p1.between_time(ts1_1, ts1_2)
        p1['returns'] = p1['close'] - p1['open']
        p2 = get_rates(s2, mt5.TIMEFRAME_H1, 50000)
        p2 = p2.between_time(ts2_1, ts2_2)
        p2['returns'] = p2['close'] - p2['open']

        pearson_corr = np.corrcoef(p1['returns'].tail(bars), p2['returns'].tail(bars)) 
        correlations[f'{ts1_1}-{ts2_1}'] = pearson_corr[0, 1]  # Store correlation

    return correlations

In [162]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def mini_corr(s1, s2, bars = 500):

    results = []

    for offset in range(24):
        for h in range(24):
            
            # start and end times for first time series
            ts1_1 = timedelta(hours=h).seconds // 3600
            ts1_2 = (ts1_1 + 1) % 24
            
            # offset hours for second time series
            ts2_1 = (ts1_1 + 1 + offset) % 24
            ts2_2 = (ts2_1 + 1) % 24
            
            p1 = get_rates(s1, mt5.TIMEFRAME_H1, 50000)
            p1 = p1.between_time(f'{ts1_1:02d}:00', f'{ts1_2:02d}:00')
            p1['returns'] = p1['close'] - p1['open']

            p2 = get_rates(s2, mt5.TIMEFRAME_H1, 50000)
            p2 = p2.between_time(f'{ts2_1:02d}:00', f'{ts2_2:02d}:00')
            p2['returns'] = p2['close'] - p2['open']

            if len(p1['returns']) > 0 and len(p2['returns']) > 0:
                pearson_corr = np.corrcoef(p1['returns'].tail(bars), p2['returns'].tail(bars))[0, 1]

                if abs(pearson_corr) > 0.5:
                    results.append((ts1_1, ts1_2, ts2_1, ts2_2, pearson_corr))
                    
    return results

In [163]:
mini_corr('AUDUSD.a', 'USDJPY.a')

[(13, 14, 13, 14, -0.5069723012444712),
 (14, 15, 14, 15, -0.7897661689149376),
 (15, 16, 15, 16, -0.7678376747086922),
 (16, 17, 16, 17, -0.582745257763089),
 (17, 18, 17, 18, -0.5154347958641352),
 (19, 20, 19, 20, -0.556167066543711),
 (20, 21, 20, 21, -0.612978471752775),
 (21, 22, 21, 22, -0.5632127428717656)]

In [167]:
def v3_mini_corr(s1, s2, bars=100):
    for offset in range(24):
        for hour in range(24):
            ts1_1 = hour
            ts1_2 = (ts1_1 + 1) % 24
            
            ts2_1 = (hour + 1 + offset) % 24
            ts2_2 = (ts2_1 + 1) % 24
            
            # Convert to string format for the 'between_time' function
            ts1_1, ts1_2, ts2_1, ts2_2 = [f'{t:02d}:00' for t in [ts1_1, ts1_2, ts2_1, ts2_2]]
            
            p1 = get_rates(s1, mt5.TIMEFRAME_H1, 50000)
            p1 = p1.between_time(ts1_1, ts1_2)
            p1['returns'] = p1['close'] - p1['open']
            
            p2 = get_rates(s2, mt5.TIMEFRAME_H1, 50000)
            p2 = p2.between_time(ts2_1, ts2_2)
            p2['returns'] = p2['close'] - p2['open']
            
            if len(p1['returns']) < bars or len(p2['returns']) < bars:
                print(f'Insufficient data for {s1} between {ts1_1} and {ts1_2} or {s2} between {ts2_1} and {ts2_2}')
                continue
            
            pearson_corr = np.corrcoef(p1['returns'].tail(bars), p2['returns'].tail(bars))[0, 1]
            
            if abs(pearson_corr) > 0.5:
                print(f'Found correlation {pearson_corr} for {s1} between {ts1_1} and {ts1_2} and {s2} between {ts2_1} and {ts2_2}')


In [168]:
v3_mini_corr('AUDUSD.a', 'USDJPY.a')

Found correlation -0.7307982835086003 for AUDUSD.a between 14:00 and 15:00 and USDJPY.a between 14:00 and 15:00
Found correlation -0.7374479429221158 for AUDUSD.a between 15:00 and 16:00 and USDJPY.a between 15:00 and 16:00
Found correlation -0.5174647667371404 for AUDUSD.a between 16:00 and 17:00 and USDJPY.a between 16:00 and 17:00


In [37]:
#Correlation for AUDJPY Syd Open - USDJPY Tokyo Open
lst = []

un_corr = []

for i in range(50):
    pearson_corr = np.corrcoef(syd_opens['Syd Pip Return'].tail(i), tokyo_opens['Tko Pip Return'].tail(i))
    if pearson_corr[0][1] > 0.5:
        # print(f'Pearson correlation {pearson_corr[0][1]} on lag {i}')
        lst.append(i)
    else:
        un_corr.append(i)
        
print(f'Lags that have a correlation higher than 0.5 are:'
      f'\n{lst}')
print(f'Total Lags: {len(lst)}')

Lags that have a correlation higher than 0.5 are:
[6, 7, 9, 10, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 33, 34, 35, 36, 39, 49]
Total Lags: 20


In [38]:
import datetime

day_counts = {}

for i in lst:
    date = syd_opens.index[i]
    day_name = date.day_name()

    if day_name not in day_counts:
        day_counts[day_name] = 0

    day_counts[day_name] += 1
print(f'Days that have correlations between AUDUSD and USDJPY of more than 0.5 are:')

for day, count in day_counts.items():
    print(f'{day}: {count}')

Days that have correlations between AUDUSD and USDJPY of more than 0.5 are:
Tuesday: 4
Wednesday: 3
Friday: 6
Monday: 5
Thursday: 2


In [39]:
import datetime

day_counts = {}

for i in un_corr:
    date = syd_opens.index[i]
    day_name = date.day_name()

    if day_name not in day_counts:
        day_counts[day_name] = 0

    day_counts[day_name] += 1
print(f'Days that do not have correlations between AUDUSD and USDJPY of more than 0.5 are:')

for day, count in day_counts.items():
    print(f'{day}: {count}')

Days that do not have correlations between AUDUSD and USDJPY of more than 0.5 are:
Monday: 5
Tuesday: 6
Wednesday: 7
Thursday: 8
Friday: 4


In [42]:
def corr_tool(symbol1, symbol2, timeframe, bars):
    print(f'Starting Pearson Correlation Analysis for {symbol1}, {symbol2} at {timeframe} for {bars} bars.')

    s1 = input(f'Start Time for {symbol1}?')
    s2 = input(f'End time for {symbol1}?')

    e1 = input(f'Start time for {symbol2}?')
    e2 = input(f'End time for {symbol2}')

    pair1 = get_rates(symbol1, timeframe, bars)
    explored_seg1 = pair1.between_time(s1, s2)
    pair2 = get_rates(symbol2,  timeframe, bars)
    explored_seg2 = pair2.between_time(e1, e2)
    
    print('Times to investigate are:'
          f'\n{s1} to {s2} and {e1} to {e2}')
    
    pair1_returns = pair1['open'] - pair1['close']
    pair2_returns = pair2['open'] - pair2['close']

    lst = []

    un_corr = []

    for i in range(50):
        pearson_corr = np.corrcoef(pair1_returns.tail(50), pair2_returns.tail(50))
        if pearson_corr[0][1] > 0.5:
            # print(f'Pearson correlation {pearson_corr[0][1]} on lag {i}')
            lst.append(i)
        else:
            un_corr.append(i)
            
    print(f'Lags that have a correlation higher than 0.5 are:'
        f'\n{lst}')
    print(f'Total Lags: {len(lst)}')

    import datetime

    day_counts = {}

    for i in lst:
        date = syd_opens.index[i]
        day_name = date.day_name()

        if day_name not in day_counts:
            day_counts[day_name] = 0

        day_counts[day_name] += 1
    print(f'Days that have correlations between {symbol1} and {symbol2}of more than 0.5 are:')

    for day, count in day_counts.items():
        print(f'{day}: {count}')
        
    day_counts = {}

    for i in un_corr:
        date = syd_opens.index[i]
        day_name = date.day_name()

        if day_name not in day_counts:
            day_counts[day_name] = 0

        day_counts[day_name] += 1
    print(f'Days that do not have correlations between {symbol1} and {symbol2} of more than 0.5 are:')

    for day, count in day_counts.items():
        print(f'{day}: {count}')

In [43]:
corr_tool('EURUSD.a', 'GBPUSD.a', mt5.TIMEFRAME_H1, 50000)

Starting Pearson Correlation Analysis for EURUSD.a, GBPUSD.a at 16385 for 50000 bars.


Start Time for EURUSD.a? 10:00
End time for EURUSD.a? 10:59
Start time for GBPUSD.a? 16:00
End time for GBPUSD.a 16:59


Times to investigate are:
10:00 to 10:59 and 16:00 to 16:59
Lags that have a correlation higher than 0.5 are:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Total Lags: 50
Days that have correlations between EURUSD.a and GBPUSD.aof more than 0.5 are:
Monday: 10
Tuesday: 10
Wednesday: 10
Thursday: 10
Friday: 10
Days that do not have correlations between EURUSD.a and GBPUSD.a of more than 0.5 are:


In [41]:
pair1 = get_rates('EURUSD.a', mt5.TIMEFRAME_H1, 10000)
pair2 = get_rates('GBPUSD.a',  mt5.TIMEFRAME_H1, 10000)